In [1]:
import json

In [2]:
with open("../../config.json") as f:
    read = json.load(f)

In [3]:
import openai
openai.api_key = read['API Key']

In [4]:
from openai import InvalidRequestError

from subprocess import check_output, STDOUT, CalledProcessError
import os

import time

In [5]:
def get_response_completion(prompt, model="code-davinci-002",temperature=0, max_tokens=100,retry=True,**wargs):
    """ Sample usage
    get_response("# program to return the nth fibbonacci number ")
    """
    try:
        response = openai.Completion.create(model=model,
                          prompt=prompt, temperature=temperature, max_tokens=max_tokens, **wargs)
    except InvalidRequestError as ie:
        time.sleep(5)
        print(ie)
        if retry:
            return get_response_completion(prompt, model=model, temperature=temperature, max_tokens=max_tokens,
                                           retry=False, **wargs)
        return None
    except Exception as e:
        print(e)
        return None
    return response

def get_response_edit(input, instruction, model="code-davinci-edit-001", temperature=0, retry=True, **wargs):
    """ Sample usage
    get_response_edit(input="def Fibonacci(n):\
	if n<0: \
		print("Incorrect input") \
	elif n==1: \
		return 0\
	elif n==2: \
		return 1\
	else: \
		return Fibonacci(n-1)+Fibonacci(n-2\
\
print(Fibonacci(9))",
                 instruction="fix the error message "+result['error'].decode("utf-8")
                 )
    """
    try:
        response = openai.Edit.create(
                  model=model,
                  input=input,
                  instruction=instruction,
                        **wargs
                )
    except InvalidRequestError as ie:
        time.sleep(10)
        print(ie)
        if retry:
            get_response_edit(input, instruction, model, retry=False, temperature=0, **wargs)
        return None

    except Exception as e:
        print(e)
        return None

    return response

In [6]:
def test_code(file_path):
    result={}
    working=True
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"this path doesn't exist: {file_path}")
    try:
        output = check_output(['python', file_path], stderr=STDOUT)
        result['out']=output.decode("utf-8")
    except CalledProcessError as exc:
        print(exc.output.decode("utf-8"))
        result['error'] = exc.output.decode("utf-8")
        working=False
    else:
        print(result['out'])
    return working,result

In [7]:
def write_to_file(text, file_path):
    with open(file_path,"w") as f:
        f.write(text)
    return True

In [8]:
def get_code(prompt, file_path="../../out/sample1.py", max_edit_count=5):
    temp_file = "../../out/temp"
    counter=1
    response = get_response_completion(prompt)
    if response is None:
        raise Exception("Failed to fetch result from API try again")
    print(response['choices'][0]['text'])
    write_to_file(response['choices'][0]['text'], file_path)
    write_to_file(response['choices'][0]['text'], temp_file+f"_{counter}.py")

    working, result = test_code(file_path)
    while not working and counter<max_edit_count:
        print(f"Correction attempt:{counter}")
        counter+=1
        response = get_response_edit(input=response['choices'][0]['text'],
                                         instruction="\nfix the error message "+result['error']
                                         )
        if response is None:
            raise Exception("Failed to fetch result from API try again")
        write_to_file(response['choices'][0]['text'], file_path)
        write_to_file(response['choices'][0]['text'], temp_file+f"_{counter}.py")
        working, result = test_code(file_path)
    

In [10]:
get_code("#program to generate 7 rows of triangle using * ", max_edit_count=10)

symbol

for i in range(7):
    for j in range(i):
        print('*',end="")
    print('')

for i in range(7,0,-1):
    for j in range(i):
        print('*',end="")
    print('')

#program to reverse a word after accepting the input from the user

word = input("Enter a word: ")


Traceback (most recent call last):
  File "/Users/harsh/Downloads/Coursework/CS 523/graphic_codex/src/Notebooks/../../out/sample1.py", line 1, in <module>
    symbol
NameError: name 'symbol' is not defined

Correction attempt:1
pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html

